In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

from tqdm import tqdm

In [ ]:
def read_data(folder):
    data = np.load(folder + '/data.npy')
    data[data < 0] = 0
    return data

In [ ]:
def make_blocks(X):
    blocks = []
    for y in range(0, np.shape(X)[1], piece_size):
        for x in range(0, np.shape(X)[2], piece_size):
          if x + piece_size > np.shape(X)[2]:
            break
          blocks.append(X[:, y : y + piece_size, x : x + piece_size])
    return torch.FloatTensor(blocks)

In [ ]:
piece_size = 64

In [ ]:
class UNET(nn.Module):
    def __init__(self, in_channels=1, classes=1):
        super(UNET, self).__init__()
        self.layers = [in_channels, 32, 64, 128, 256, 512]

        self.double_conv_downs = nn.ModuleList(
            [self.__double_conv(layer, layer_n) for layer, layer_n in zip(self.layers[:-1], self.layers[1:])])

        self.up_trans = nn.ModuleList(
            [nn.ConvTranspose3d(layer, layer_n, kernel_size=2, stride=2)
             for layer, layer_n in zip(self.layers[::-1][:-2], self.layers[::-1][1:-1])])

        self.double_conv_ups = nn.ModuleList(
        [self.__double_conv(layer, layer//2) for layer in self.layers[::-1][:-2]])

        self.max_pool_2x2 = nn.MaxPool3d(kernel_size=2, stride=2)

        self.final_conv = nn.Conv3d(32, classes, kernel_size=1)


    def __double_conv(self, in_channels, out_channels):
        conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )
        return conv

    def forward(self, x):
        # down layers
        concat_layers = []

        for down in self.double_conv_downs:
            x = down(x)
            if down != self.double_conv_downs[-1]:
                concat_layers.append(x)
                x = self.max_pool_2x2(x)

        concat_layers = concat_layers[::-1]

        # up layers
        for up_trans, double_conv_up, concat_layer  in zip(self.up_trans, self.double_conv_ups, concat_layers):
            x = up_trans(x)
            if x.shape != concat_layer.shape:
                x = TF.resize(x, concat_layer.shape[2:])

            concatenated = torch.cat((concat_layer, x), dim=1)
            x = double_conv_up(concatenated)

        x = self.final_conv(x)

        return x

In [ ]:
model = UNET(1, 5).to('cuda')

In [ ]:
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params


# Подсчитываем количество параметров
total_params, trainable_params = count_parameters(model)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Non-trainable parameters: {total_params - trainable_params}")

Total parameters: 22576933
Trainable parameters: 22576933
Non-trainable parameters: 0


## Проверка модели с 4 эпохами

## 2 cross val

In [ ]:
def mask_from_model(num):
    model = UNET(1, 5).to('cuda')
    model.load_state_dict(torch.load('C:/Users/MK/Nika/exp5/hyperparameter2/unet3d'+ num, map_location=torch.device('cuda')))
    model.eval()

    data = np.load('C:/Users/MK/Nika/Data64/finger'+ num + '_data.npy')

    blocks1 = torch.load('C:/Users/MK/Nika/Data64/finger'+ num + '.pt')
    blocks2 = torch.load('C:/Users/MK/Nika/Data64/finger'+ num + '_2.pt')
    blocks1 = blocks1.to('cuda')
    blocks2 = blocks2.to('cuda')

    out_model = np.zeros((blocks1.size()[0], 5, 256, piece_size, piece_size))
    for i in range(blocks1.size()[0]):
      out_model[i] = model(blocks1[i].unsqueeze(0).unsqueeze(0).to('cuda')).detach().cpu()
    in_new = np.zeros((blocks2.size()[0], 5, 256, piece_size, piece_size))
    for i in tqdm(range(blocks2.size()[0])):
      in_new[i] = model(blocks2[i].unsqueeze(0).unsqueeze(0).to('cuda')).detach().cpu()
    data_out = np.zeros((5, 256, data.shape[1], data.shape[2]))
    for y_step in range(data.shape[1]//piece_size):
      for x_step in range(data.shape[2]//piece_size):
        x_begin = x_step*piece_size
        x_end = x_step*piece_size + piece_size
        y_begin = y_step*piece_size
        y_end = y_step*piece_size+piece_size
        num_block = x_step+y_step*(data.shape[2]//piece_size)
        data_out[:,:, y_begin:y_end, x_begin:x_end] = out_model[num_block, :,:,:,:]
    data_out_new = np.zeros((5, 256, np.shape(data)[1]-piece_size, np.shape(data)[2]-piece_size))
    for y_step in range(data.shape[1]//piece_size-1):
      for x_step in range(data.shape[2]//piece_size-1):
        x_begin = x_step*piece_size
        x_end = x_step*piece_size + piece_size
        y_begin = y_step*piece_size
        y_end = y_step*piece_size+piece_size
        num_block = x_step+y_step*(data.shape[2]//piece_size-1)
        data_out_new[:,:, y_begin:y_end,x_begin:x_end] = in_new[num_block, :,:,:,:]
    for x in range(piece_size//2, data.shape[2] - piece_size//2):
      for y in range(piece_size//2, data.shape[1] - piece_size//2):
        data_out[:, :, y, x] += data_out_new[:, :, y-piece_size//2, x-piece_size//2]
        data_out[:, :, y, x] /= 2
    masks_out_with_bags = np.zeros((5, 256, data_out.shape[2], data_out.shape[3]))
    for x in tqdm(range(data_out.shape[3])):
      for y in range(data_out.shape[2]):
        for z in range(256):
          ind = np.argmax(data_out[:, z, y, x])
          masks_out_with_bags[ind, z, y, x] = 1
    np.save('C:/Users/MK/Nika/exp5/hyperparameter2/val/masks_model'+ num + '.npy', masks_out_with_bags)

In [ ]:
mask_from_model(str(0))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:26<00:00,  2.97it/s]


In [ ]:
mask_from_model(str(1))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:25<00:00,  2.98it/s]


In [ ]:
mask_from_model(str(2))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:24<00:00,  3.05it/s]


In [ ]:
mask_from_model(str(3))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:21<00:00,  3.14it/s]


In [ ]:
mask_from_model(str(4))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:23<00:00,  3.07it/s]


# Эксперимент с 40 эпохами

## 2 cross val

In [ ]:
def mask_from_model(num):
    model = UNET(1, 5).to('cuda')
    model.load_state_dict(torch.load('C:/Users/MK/Nika/exp5/hyperparameter3/unet3d'+ num, map_location=torch.device('cuda')))
    model.eval()

    data = np.load('C:/Users/MK/Nika/Data64/finger'+ num + '_data.npy')

    blocks1 = torch.load('C:/Users/MK/Nika/Data64/finger'+ num + '.pt')
    blocks2 = torch.load('C:/Users/MK/Nika/Data64/finger'+ num + '_2.pt')
    blocks1 = blocks1.to('cuda')
    blocks2 = blocks2.to('cuda')

    out_model = np.zeros((blocks1.size()[0], 5, 256, piece_size, piece_size))
    for i in range(blocks1.size()[0]):
      out_model[i] = model(blocks1[i].unsqueeze(0).unsqueeze(0).to('cuda')).detach().cpu()
    in_new = np.zeros((blocks2.size()[0], 5, 256, piece_size, piece_size))
    for i in tqdm(range(blocks2.size()[0])):
      in_new[i] = model(blocks2[i].unsqueeze(0).unsqueeze(0).to('cuda')).detach().cpu()
    data_out = np.zeros((5, 256, data.shape[1], data.shape[2]))
    for y_step in range(data.shape[1]//piece_size):
      for x_step in range(data.shape[2]//piece_size):
        x_begin = x_step*piece_size
        x_end = x_step*piece_size + piece_size
        y_begin = y_step*piece_size
        y_end = y_step*piece_size+piece_size
        num_block = x_step+y_step*(data.shape[2]//piece_size)
        data_out[:,:, y_begin:y_end, x_begin:x_end] = out_model[num_block, :,:,:,:]
    data_out_new = np.zeros((5, 256, np.shape(data)[1]-piece_size, np.shape(data)[2]-piece_size))
    for y_step in range(data.shape[1]//piece_size-1):
      for x_step in range(data.shape[2]//piece_size-1):
        x_begin = x_step*piece_size
        x_end = x_step*piece_size + piece_size
        y_begin = y_step*piece_size
        y_end = y_step*piece_size+piece_size
        num_block = x_step+y_step*(data.shape[2]//piece_size-1)
        data_out_new[:,:, y_begin:y_end,x_begin:x_end] = in_new[num_block, :,:,:,:]
    for x in range(piece_size//2, data.shape[2] - piece_size//2):
      for y in range(piece_size//2, data.shape[1] - piece_size//2):
        data_out[:, :, y, x] += data_out_new[:, :, y-piece_size//2, x-piece_size//2]
        data_out[:, :, y, x] /= 2
    masks_out_with_bags = np.zeros((5, 256, data_out.shape[2], data_out.shape[3]))
    for x in tqdm(range(data_out.shape[3])):
      for y in range(data_out.shape[2]):
        for z in range(256):
          ind = np.argmax(data_out[:, z, y, x])
          masks_out_with_bags[ind, z, y, x] = 1
    np.save('C:/Users/MK/Nika/exp5/hyperparameter3/val/masks_model'+ num + '.npy', masks_out_with_bags)

In [ ]:
mask_from_model(str(0))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:22<00:00,  3.09it/s]


In [ ]:
mask_from_model(str(1))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:23<00:00,  3.08it/s]


In [ ]:
mask_from_model(str(2))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:24<00:00,  3.04it/s]


In [ ]:
mask_from_model(str(3))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:22<00:00,  3.11it/s]


In [ ]:
mask_from_model(str(4))

100%|████████████████████████████████████████████████████████████████████████████████| 256/256 [01:22<00:00,  3.10it/s]
